# SQL Server Window Functions

SQL Server Window Functions calculate an aggregate value based on a group of rows and return multiple rows for each group.  

- SUM()
  
  
- MAX():


- MIN():


- AVG(): 


- COUNT():


- DENSE_RANK: Assign a rank value to each row within a partition of a result, with no gaps in rank values.


- RANK: Assign a rank value to each row within a partition of a result set.


- ROW_NUMBER: Assign a unique sequential integer to rows within a partition of a result set, the first row starts from 1.


- PERCENT_RANK: Calculate the percent rank of a value in a set of values.


- FIRST_VALUE: Get the value of the first row in an ordered partition of a result set.


- LAST_VALUE: Get the value of the last row in an ordered partition of a result set.


- LAG: Provide access to a row at a given physical offset that comes before the current row.


- LEAD: Provide access to a row at a given physical offset that follows the current row.


- NTILE: Distribute rows of an ordered partition into a number of groups or buckets.


- CUME_DIST: Calculate the cumulative distribution of a value in a set of values.


## SQL Server DENSE_RANK Function

The DENSE_RANK() is a window function that assigns a rank value to each row within a partition of a result, with no gaps in rank values.  

**Note:** Unlike the RANK() function, the DENSE_RANK() function returns consecutive rank values.Rows in each partition receive the same ranks if they have the same values.    

The syntax of the DENSE_RANK() function is as follows:  

```sql
DENSE_RANK() OVER (
    [PARTITION BY partition_expression, ... ]
    ORDER BY sort_expression [ASC | DESC], ...
)
```  

**SQL Server DENSE_RANK() function illustration**

The following statements create a new table named dense_rank_demo and insert some rows into that table:  

```sql
CREATE TABLE sales.dense_rank_demo (
	v VARCHAR(10)
);
	
INSERT INTO sales.dense_rank_demo(v)
VALUES('A'),('B'),('B'),('C'),('C'),('D'),('E');
	
SELECT * FROM sales.dense_rank_demo;

SELECT
	v,
	DENSE_RANK() OVER (
		ORDER BY v
	) my_dense_rank,
	RANK() OVER (
		ORDER BY v
	) my_rank
FROM
	sales.dense_rank_demo;
```  

**SQL Server DENSE_RANK() function examples**  

The following statement ranks products in each category by list prices. It returns only the top 3 products per category by list prices.  

```sql
SELECT * FROM (
	SELECT
		product_id,
		product_name,
		category_id,
		list_price,
		DENSE_RANK () OVER ( 
			PARTITION BY category_id
			ORDER BY list_price DESC
		) price_rank 
	FROM
		production.products
) t
WHERE price_rank <= 3;
```  

## SQL Server RANK Function

The RANK() function is a window function that calculates a rank for each row within a partition of a result set.The rows within a partition that have the same values will receive the same rank. The rank of the first row within a partition is one.     

**Note:** The RANK() function adds the number of tied rows to the tied rank to calculate the rank of the next row, therefore, the ranks may not be consecutive.    

The following shows the syntax of the RANK() function:

```sql
RANK() OVER (
    [PARTITION BY partition_expression, ... ]
    ORDER BY sort_expression [ASC | DESC], ...
)
```  

The RANK() function is useful for **top-N** and **bottom-N** reports.   

**SQL Server RANK() function examples**   

This example uses the RANK() function to assign a rank to each product by list price in each brand and returns products with rank less than or equal to three:
    
```sql
SELECT * FROM (
	SELECT
		product_id,
		product_name,
		brand_id,
		list_price,
		RANK () OVER ( 
			PARTITION BY brand_id
			ORDER BY list_price DESC
		) price_rank 
	FROM
		production.products
) t
WHERE price_rank <= 3;
```

## SQL Server ROW_NUMBER Function

The ROW_NUMBER() is a window function that assigns a sequential integer to each row within the partition of a result set. The row number starts with 1 for the first row in each partition.   

The following shows the syntax of the ROW_NUMBER() function:   

```sql
ROW_NUMBER() OVER (
    [PARTITION BY partition_expression, ... ]
    ORDER BY sort_expression [ASC | DESC], ...
)
```
**Using SQL Server ROW_NUMBER() for pagination**   

The ROW_NUMBER() function is useful for pagination in applications. For example, you can display a list of customers by page, where each page has 10 rows.   

The following example uses the ROW_NUMBER() to return customers from row 11 to 20, which is the second page:   

```sql
WITH cte_customers AS (
    SELECT 
        ROW_NUMBER() OVER(
             ORDER BY 
                first_name, 
                last_name
        ) row_num, 
        customer_id, 
        first_name, 
        last_name
     FROM 
        sales.customers
) SELECT 
    customer_id, 
    first_name, 
    last_name
FROM 
    cte_customers
WHERE 
    row_num > 20 AND 
    row_num <= 30;
```

## SQL Server PERCENT_RANK Function

The PERCENT_RANK() function is similar to the CUME_DIST() function. The PERCENT_RANK() function evaluates the relative rank of a row within a group of rows.   

The following illustrates the syntax of the SQL Server PERCENT_RANK() function:   

```sql
PERCENT_RANK() OVER (
    [PARTITION BY partition_expression, ... ]
    ORDER BY sort_expression [ASC | DESC], ...
)
```
- The result of PERCENT_RANK() is greater than 0 and less than or equal to 1.
- The first row has a rank value of zero. Tie values evaluate to the same cumulative distribution value.
- The PERCENT_RANK() function includes NULL values by default and treats them as the lowest possible values.

**SQL Server PERCENT_RANK() examples**

The following statement creates a new view named sales.vw_staff_sales for the demonstration.   

```sql
CREATE VIEW sales.vw_staff_sales(
    staff_id, 
    year, 
    net_sales
) AS
SELECT 
    staff_id, 
    YEAR(order_date), 
    ROUND(SUM(quantity*list_price*(1-discount)),0)
FROM 
    sales.orders o
INNER JOIN sales.order_items i on i.order_id = o.order_id
WHERE 
    staff_id IS NOT NULL
GROUP BY 
    staff_id, 
    YEAR(order_date);
```

The following example uses the PERCENT_RANK() to calculate the sales percentile for each staff in 2016 and 2017.  

```sql
SELECT 
    year,
    CONCAT_WS(' ',first_name,last_name) full_name,
    net_sales, 
    FORMAT(
        PERCENT_RANK() OVER (
            PARTITION BY year
            ORDER BY net_sales DESC
        ) ,
    'P') percent_rank

FROM 
    sales.vw_staff_sales t
INNER JOIN sales.staffs m on m.staff_id = t.staff_id
WHERE 
    YEAR IN (2016,2017);
```

## SQL Server FIRST_VALUE Function

The FIRST_VALUE() function is a window function that returns the first value in an ordered partition of a result set.   

The following shows the syntax of the FIRST_VALUE() function:   

```sql
FIRST_VALUE ( scalar_expression )  
OVER ( 
    [PARTITION BY partition_expression, ... ]
    ORDER BY sort_expression [ASC | DESC], ...
)
```

- The scalar_expression can be a column, subquery, or expression that evaluates to a single value. It cannot be a window function.
- The rows_range_clause further limits the rows within the partition by defining start and end points.

**SQL Server FIRST_VALUE() function examples**   

The following example uses the FIRST_VALUE() function to return product categories with the lowest sales volumes in 2016 and 2017.   

```sql
SELECT 
    category_name,
    year,
    qty,
    FIRST_VALUE(category_name) OVER(
        `PARTITION BY` year
        ORDER BY qty
    ) lowest_sales_volume
FROM 
    sales.vw_category_sales_volume
WHERE
    year BETWEEN 2016 AND 2017;
```

## SQL Server LAST_VALUE Function

The LAST_VALUE() function is a window function that returns the last value in an ordered partition of a result set.  

The following shows the syntax of the LAST_VALUE() function:   

```sql
LAST_VALUE ( scalar_expression )  
OVER ( 
    [PARTITION BY partition_expression, ... ]
    ORDER BY sort_expression [ASC | DESC], ...
)    
```

**SQL Server LAST_VALUE() function examples**   

The following example uses the LAST_VALUE() function to return product categories with the highest sales volumes in 2016 and 2017.   

```sql
SELECT 
    category_name,
    year,
    qty,
    LAST_VALUE(category_name) OVER(
			PARTITION BY year
        ORDER BY qty
        RANGE BETWEEN 
            UNBOUNDED PRECEDING AND 
            UNBOUNDED FOLLOWING
    ) highest_sales_volume
FROM 
    sales.vw_category_sales_volume
WHERE
    year IN (2016,2017);
```

- The RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING clause defined the frame in the partition starting from the first row and ending at the last row.


- With the ROWS option you define on a physical level how many rows are included in your window frame. With the RANGE option how many rows are included in the window frame depends on the ORDER BY values.    

## SQL Server LAG Function

SQL Server LAG() is a window function that provides access to a row at a specified physical offset which comes before the current row.   

In other words, by using the LAG() function, from the current row, you can access data of the previous row, or the row before the previous row, and so on.   

The LAG() function can be very useful for comparing the value of the current row with the value of the previous row.   

The following shows the syntax of the LAG() function:   

```sql
LAG(return_value ,offset [,default]) 
OVER (
    [PARTITION BY partition_expression, ... ]
    ORDER BY sort_expression [ASC | DESC], ...
)
```

**SQL Server LAG() function examples**

To compare the sales of the current month with the previous month of net sales by brand in 2018, you use the following query:  

```sql
WITH cte_sales AS (
	SELECT 
		month,
		brand_name,
		net_sales,
		LAG(net_sales,1) OVER (
			PARTITION BY brand_name
			ORDER BY month
		) previous_sales
	FROM 
		sales.vw_netsales_brands
	WHERE
		year = 2018
)
SELECT 
	month, 
	brand_name,
	net_sales, 
	previous_sales,
	FORMAT(
		(net_sales - previous_sales)  / previous_sales,
		'P'
	) vs_previous_month
FROM
	cte_sales;
```

## SQL Server LEAD Function

SQL Server LEAD() is a window function that provides access to a row at a specified physical offset which follows the current row.   

For example, by using the LEAD() function, from the current row, you can access data of the next row, or the row after the next row, and so on.   

The LEAD() function can be very useful for comparing the value of the current row with the value of the following row.   

The following shows the syntax of the LEAD() function:   

```sql
LEAD(return_value ,offset [,default]) 
OVER (
    [PARTITION BY partition_expression, ... ]
    ORDER BY sort_expression [ASC | DESC], ...
)
```
**SQL Server LEAD() function examples**
Let’s create a new view named sales.vw_netsales_brands for the demonstration:   

```sql
CREATE VIEW sales.vw_netsales_brands
AS
	SELECT 
		c.brand_name, 
		MONTH(o.order_date) month, 
		YEAR(o.order_date) year, 
		CONVERT(DEC(10, 0), SUM((i.list_price * i.quantity) * (1 - i.discount))) AS net_sales
	FROM sales.orders AS o
		INNER JOIN sales.order_items AS i ON i.order_id = o.order_id
		INNER JOIN production.products AS p ON p.product_id = i.product_id
		INNER JOIN production.brands AS c ON c.brand_id = p.brand_id
	GROUP BY c.brand_name, 
			MONTH(o.order_date), 
			YEAR(o.order_date)
```

To compare the sales of the current month with the previous month of net sales by brand in 2018, you use the following query:   

```sql
WITH cte_sales AS (
	SELECT 
		month,
		brand_name,
		net_sales,
		LAG(net_sales,1) OVER (
			PARTITION BY brand_name
			ORDER BY month
		) previous_sales
	FROM 
		sales.vw_netsales_brands
	WHERE
		year = 2018
)
SELECT 
	month, 
	brand_name,
	net_sales, 
	previous_sales,
	FORMAT(
		(net_sales - previous_sales)  / previous_sales,
		'P'
	) vs_previous_month
FROM
	cte_sales;
```

## SQL Server NTILE Function

The SQL Server NTILE() is a window function that distributes rows of an ordered partition into a specified number of approximately equal groups, or buckets.    

It assigns each group a bucket number starting from one. For each row in a group, the NTILE() function assigns a bucket number representing the group to which the row belongs.   

The syntax of the NTILE() function is as follows:   

```sql
NTILE(buckets) OVER (
    [PARTITION BY partition_expression, ... ]
    ORDER BY sort_expression [ASC | DESC], ...
)
```

-  The buckets can be an expression or subquery that evaluates to a positive integer. It cannot be a window function.   

**SQL Server NTILE() function examples**   

The following statement creates a view that returns the net sales in 2017 by months.   

```sql
CREATE VIEW sales.vw_netsales_2017 AS
SELECT 
	c.category_name,
	DATENAME(month, o.shipped_date) month, 
	CONVERT(DEC(10, 0), SUM(i.list_price * quantity * (1 - discount))) net_sales
FROM 
	sales.orders o
INNER JOIN sales.order_items i ON i.order_id = o.order_id
INNER JOIN production.products p on p.product_id = i.product_id
INNER JOIN production.categories c on c.category_id = p.category_id
WHERE 
	YEAR(shipped_date) = 2017
GROUP BY
	c.category_name,
	DATENAME(month, o.shipped_date);
```

This example uses the NTILE() function to divide the net sales by month into 4 groups for each product category:     

```sql
SELECT
	category_name,
	month, 
	FORMAT(net_sales,'C','en-US') net_sales,
	NTILE(4) OVER(
		PARTITION BY category_name
		ORDER BY net_sales DESC
	) net_sales_group
FROM 
	sales.vw_netsales_2017;
```

## SQL Server CUME_DIST Function

The CUME_DIST() function calculates the cumulative distribution of a value within a group of values. Simply put, it calculates the relative position of a value in a group of values.

Sometimes, you want to make a report that contains the top or bottom x% values from a data set e.g., top 5% sales staffs by net sales. One way to achieve this with SQL Server is to use the CUME_DIST() function.   

The following shows the syntax of the CUME_DIST() function:   

```sql
CUME_DIST() OVER (
    [PARTITION BY partition_expression, ... ]
    ORDER BY sort_expression [ASC | DESC], ...
)
```

- The result of CUME_DIST() is greater than 0 and less than or equal to 1.   
- The function returns the same cumulative distribution values for the same tie values. 


**SQL Server CUME_DIST() examples**

To get the top 20% of sales staff by net sales in 2016 and 2017, you use the following query:   

```sql
WITH cte_sales AS (
    SELECT 
        CONCAT_WS(' ',first_name,last_name) full_name,
        net_sales, 
        year,
        CUME_DIST() OVER (
            PARTITION BY year
            ORDER BY net_sales DESC
        ) cume_dist
    FROM 
        sales.vw_staff_sales t
        INNER JOIN sales.staffs m  
            ON m.staff_id = t.staff_id
    WHERE 
        year IN (2016,2017)
)
SELECT 
    * 
FROM 
    cte_sales
WHERE 
    cume_dist <= 0.20;
```